### Confounding

Confounding is the phenomenon whereby coefficients can be estimated in a misleading way due to the absence of highly relevant predictive factors. 

For example, suppose we were interested in how smoking affects lung capacity in children. If we were to fit a linear model of lung capacity against rate of smoking, we would likely observe that smoking has a positive correlation with lung capacity. This may lead us to the false conclusion that children who smoke more are more likely to have larger lungs. However we have missed off a highly relevant predictor, namely age. Age is both highly correlated with lung capacity and smoking rate. Indeed, older children will tend to have larger lungs and be more likely to smoke. So if we refit our model, this time also including age as a predictor, we will likely see a different result. That is, after accounting for age, smoking will be associated with smaller lung capacities. 

Confounding leads to biased or misleading estimates of effect sizes and directions, often due to omitted variables, and must be carefully controlled for to avoid false conclusions. It is difficult to directly prevent confounding but the following may help:
- Domain knowledge: Identify potential confounders before modeling (e.g., age, gender, socioeconomic status in medical studies).
- Study design: Randomised controlled trials are designed to eliminate confounding by randomly assigning treatment.


